In [1]:
import pandas as pd

In [ ]:
# input file name
input_file='data.csv'

# load the input dataset into a pandas dataframe
df = pd.read_csv(input_file)

# select only product and description column
df = df[["product", "description"]] 
print(f"Number of Products: {len(df)}")
df.head()

In [15]:
# import torch
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List
import pandas as pd

ModuleNotFoundError: No module named 'qdrant_client'

In [16]:
input_file='data.csv'

In [14]:
# load the input dataset into a pandas dataframe
df = pd.read_csv(input_file)
df = df[["product", "description"]]  # select only product and description column

print(f"Number of Products: {len(df)}")
df.head()

Number of Products: 27555


,product,description
0,Garlic Oil - Vegetarian Capsule 500 mg,This Product contains Garlic Oil that is known...
1,Water Bottle - Orange,"Each product is microwave safe (without lid), ..."
2,"Brass Angle Deep - Plain, No.2","A perfect gift for all occasions, be it your m..."
3,Cereal Flip Lid Container/Storage Jar - Assort...,Multipurpose container with an attractive desi...
4,Creme Soft Soap - For Hands & Body,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
client = QdrantClient(":memory:")

In [ ]:
collection_name = "query-engine-products"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(    # Vector Params to be revisited
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

### Embedding model

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever. It's also quite competitive on two embedding and retrieval benchmarks: [MTEB](https://github.com/embeddings-benchmark/mteb) and [BEIR](arxiv.org/abs/2104.08663)

In [ ]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

In [ ]:
batch_size = 512  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for index in tqdm(range(0, len(df), batch_size)):
    i_end = min(index + batch_size, len(df))  # find end of batch
    batch = df.iloc[index:i_end]  # extract batch
    emb = retriever.encode(batch["plot"].tolist()).tolist()  # generate embeddings for batch
    meta = batch.to_dict(orient="records")  # get metadata
    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(df)

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

In [ ]:
def get_relevant_plot(question: str, top_k: int) -> List[str]:
    """
    Get the relevant plot for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = retriever.encode(question).tolist()  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [x.payload["title"], x.payload["plot"]] for x in result
        ]  # extract title and payload from result
        return context

    except Exception as e:
        print({e})

In [ ]:
def extract_answer(question: str, context: List[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c[1])

        # add the context to answer dict for printing both together, we print only first 500 characters of plot
        answer["title"] = c[0]
        results.append(answer)

    # sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        print(f"{i+1}", end=" ")
        print(
            "Answer: ",
            sorted_result[i]["answer"],
            "\n  Title: ",
            sorted_result[i]["title"],
            "\n  score: ",
            sorted_result[i]["score"],
        )